# Script permettant de récupérer les séquences genomiques/CDS des transcript d'intéret (sequence/id) ainsi que les positiosn des exons (lookup/id)

Input: fichier transcript_ensembl.tab tableau de correspondance Uniprot AC/ID -> Ensembl Transcript téléchargé sur https://www.uniprot.org/uploadlists/  
Output:  
    genomic_new.fa / cds_new.fa fichier contenant toute les séquences fasta (génomique ou CDS) pour nos protéines d'intérêt  
    lookup_newline.json fichier JSON contenant les coordonées exonique de toute les séquences à traiter pour générer les exon map.

In [1]:
# Importation des lib et data 

import pandas as pd
import requests, sys
import grequests
import json
ID_file = pd.read_csv("../raw/uniprot-exon-map/transcript_ensembl.tab", sep = "\t")

In [3]:
# Création des requêtes parallèles sur les serveur de ensembl pour récupérer les séquences génomique ou CDS (changer type: genomic ou cds)
# Peut etre assez long (1h - 1H30), changer le paramètre size pour faire plus de requête simultanée
# Check en printant all_response si toute les réponses sont 200, dans le cas contraire certaines données sont perdues / sont à refaire
url = "https://rest.ensembl.org/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

params = []
for i in range(0, 9960, 50):
    try:
        params.append({"ids": ID_file.iloc[i:i+50,1].tolist(), "type":"genomic"})
    except:
        params.append({"ids": ID_file.iloc[i:,1].tolist(), "type":"genomic"})

        
rs = [grequests.post(url, headers=headers, data=json.dumps(i)) for i in params]
all_response = grequests.map(rs, size=50)

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
# Ecriture des résultats des réponses dans un fichier fasta. Penser à changer le nom du fichier et le header par le bon type CDS/GENOMIC
f = open("../raw/uniprot-exon-map/genomic_new.fa", "w")
indice = 0
for response in all_response:
    j = 0
    try:
        for entry in response.json():
            f.write(">"+entry['query']+" "+str(ID_file.iloc[indice*50+j,0]) + " GENOMIC\n")
            f.write(entry['seq']+"\n")
            j = j+1
    except:
        pass
    indice = indice + 1

f.close()

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
# Création des requêtes parallèles sur les serveur de ensembl pour récupérer les séquences informations de localisation des exons
# Peut etre assez long (1h)
# Check en printant all_response si toute les réponses sont 200, dans le cas contraire certaines données sont perdues / sont à refaire
url = "https://rest.ensembl.org/lookup/id/"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

params = []
for i in range(0, 9960, 1000):
    try:
        params.append({"ids": ID_file.iloc[i:i+1000,1].tolist(), "expand":"1"})
    except:
        params.append({"ids": ID_file.iloc[i:,1].tolist(), "expand":"1"})

rs = [grequests.post(url, headers=headers, data=json.dumps(i)) for i in params]
all_response = grequests.map(rs)

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [67]:
# Ecriture des résultats des réponses dans un fichier json.
f = open("../uniprot-to-exon/lookup_newline.json", "a")
for response in all_response_V2:
    try:
        f.write(json.dumps(response.json())+"\n")
    except:
        pass
f.close()